In [1]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2

In [2]:
savedir = "./nets"

In [5]:
n_input  = [64,64]
n_output = 8
weights  = {
    'wc1': tf.Variable(tf.random_normal([3, 3, 1, 128], stddev=0.1)),
    'wd1': tf.Variable(tf.random_normal(32*32*128, n_output], stddev=0.1))
}
biases   = {
    'bc1': tf.Variable(tf.random_normal([128], stddev=0.1)),
    'bd1': tf.Variable(tf.random_normal([n_output], stddev=0.1))
}
# def conv_simple(_input, _w, _b):
#     # Reshape input
#     _input_r = tf.reshape(_input, shape=[-1, 28, 28, 1])
#     # Convolution
#     _conv1 = tf.nn.conv2d(_input_r, _w['wc1'], strides=[1, 1, 1, 1], padding='SAME')
#     # Add-bias
#     _conv2 = tf.nn.bias_add(_conv1, _b['bc1'])
#     # Pass ReLu
#     _conv3 = tf.nn.relu(_conv2)
#     # Max-pooling
#     _pool  = tf.nn.max_pool(_conv3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
#     # Vectorize
#     _dense = tf.reshape(_pool, [-1, _w['wd1'].get_shape().as_list()[0]])
#     # Fully-connected layer
#     _out = tf.add(tf.matmul(_dense, _w['wd1']), _b['bd1'])
#     # Return everything
#     out = {
#         'input_r': _input_r, 'conv1': _conv1, 'conv2': _conv2, 'conv3': _conv3
#         , 'pool': _pool, 'dense': _dense, 'out': _out
#     }
    
#     return out


test_img = cv2.imread("./dataset/anger/0.jpg")
#    _pred = conv_simple(x, weights, biases)['out']
x = tf.placeholder(tf.float32, n_input)
#y = tf.placeholder(tf.float32, [None, n_output])
#output = conv_simple(x, weights, biases)['out']

# Reshape input
_input_r = tf.reshape(x, shape=[-1, 64, 64, 1])
# Convolution
_conv1 = tf.nn.conv2d(_input_r, weights['wc1'], strides=[1, 1, 1, 1], padding='SAME')
# Add-bias
_conv2 = tf.nn.bias_add(_conv1, biases['bc1'])
# Pass ReLu
_conv3 = tf.nn.relu(_conv2)
# Max-pooling
_pool  = tf.nn.max_pool(_conv3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
# Vectorize
_dense = tf.reshape(_pool, [-1, weights['wd1'].get_shape().as_list()[0]])
# Fully-connected layer
_out = tf.add(tf.matmul(_dense, weights['wd1']), biases['bd1'])
#output = tf.nn.softmax_cross_entropy_with_logits(_dense, weights['wd1'], biases['bd1'])
# Return everything

out = {
    'input_r': _input_r, 'conv1': _conv1, 'conv2': _conv2, 'conv3': _conv3
    , 'pool': _pool, 'dense': _dense, 'out': _out
}
    

saver = tf.train.Saver()

In [6]:
test_img.shape

def rgb2gray(rgb):
    if len(rgb.shape) is 3:
        print("Image is not Gray")
        return cv2.cvtColor(rgb, cv2.COLOR_BGR2GRAY)
    else:
        print("Current Image if Gray")
        return rgb

imgsize = [64, 64]
grayimg = rgb2gray(test_img)
graysmall = cv2.resize(grayimg, (imgsize[0], imgsize[1]), interpolation = cv2.INTER_CUBIC) / 255. #Normalization
#grayvec = np.reshape(graysmall, (1, -1))
graysmall.shape

Image is not Gray


(28, 28)

In [13]:
with tf.Session() as sess:
    # Restore variables from disk.
    saver.restore(sess, "./nets/cnn_mnist_simple.ckpt-4")
    feed_dict={x: graysmall}
    print "emotion is ", sess.run(_out, feed_dict )
    
    print("Model restored.")

NotFoundError: Tensor name "Variable_9" not found in checkpoint files ./nets/cnn_mnist_simple.ckpt-4
	 [[Node: save_1/restore_slice_11 = RestoreSlice[dt=DT_FLOAT, preferred_shard=-1, _device="/job:localhost/replica:0/task:0/cpu:0"](_recv_save_1/Const_0, save_1/restore_slice_11/tensor_name, save_1/restore_slice_11/shape_and_slice)]]
Caused by op u'save_1/restore_slice_11', defined at:
  File "/Users/ryan/anaconda/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/Users/ryan/anaconda/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Users/ryan/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/ryan/anaconda/lib/python2.7/site-packages/traitlets/config/application.py", line 596, in launch_instance
    app.start()
  File "/Users/ryan/anaconda/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 442, in start
    ioloop.I

In [ ]:
# Parameters
learning_rate   = 0.001
training_epochs = 5
batch_size      = 100
display_step    = 1
# Functions! 
with tf.device(device_type): # <= This is optional
    _pred = conv_simple(x, weights, biases)['out']
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(_pred, y))
    optm = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    _corr = tf.equal(tf.argmax(_pred,1), tf.argmax(y,1)) # Count corrects
    accr = tf.reduce_mean(tf.cast(_corr, tf.float32)) # Accuracy
    init = tf.initialize_all_variables()


In [10]:
training_epochs = 5
saver.restore(sess, "nets/cnn_mnist_simple.ckpt-" + str(epoch))
print ("NETWORK RESTORED")

NameError: name 'saver' is not defined

In [13]:
with tf.Session() as sess:
    # Restore variables from disk.
    tf.train.Saver().restore(sess, "./nets/checkpoint")
    print("Model restored.")

ValueError: No variables to save